# Environment Setting

In [1]:
import os
os.environ['SDK_CLIENT_HOST'] = 'https://pre-engine-aiearth.aliyun.com'
import aie

aie.Authenticate("5a15bf07bb40c4d35c1429b22bb3245a")
aie.g_var.set_var(aie.g_var.GVarKey.Log.LOG_LEVEL, aie.g_var.LogLevel.INFO_LEVEL)

In [2]:
work_dir = "./work_dirs/tutorial"

# Dataset
## Pre-defined dataset

In [3]:
# 样本集
from aietorch.datasets.aie.aie_dataset import BinaryChangeDetDataset, LandcoverDataset
myDataSet = BinaryChangeDetDataset(276, data_root=work_dir)

requelt sampletset info start
https://pre-engine-aiearth.aliyun.com/trainSDK/api/sdk/sampleset/dinfo?id=276
requelt sampletset info end
root_dir:./work_dirs/tutorial/276/ipwzhdguqhypapsw
preprecess start
preprecess end


/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
landcover = LandcoverDataset(349, data_root=work_dir)

requelt sampletset info start
https://pre-engine-aiearth.aliyun.com/trainSDK/api/sdk/sampleset/dinfo?id=349
requelt sampletset info end
root_dir:./work_dirs/tutorial/349/fonnnyvsopwfkrqe
preprecess start
preprecess end


In [5]:
# 提取单个类目，其他类目作为背景
landcover.set_classes_filter(['industrial_land'])

preprecess start
preprecess end


In [6]:
landcover.classes

['background', 'industrial_land']

In [7]:
landcover.restore_classes()

preprecess start
preprecess end


In [8]:
landcover.classes

['背景',
 'industrial_land',
 'garden_land',
 'urban_residential',
 'arbor_forest',
 'rural_residential',
 'shrub_land',
 'traffic_land',
 'natural_meadow',
 'paddy_field',
 'artificial_meadow',
 'irrigated_land',
 'river',
 'dry_cropland',
 'lake',
 'pond']

In [9]:
from aietorch.sampler import RandomNonGeoDatasetSampler


In [13]:
# 随机按照80%， 20%进行切分成两个新样本集
train_dataset, val_dataset = RandomNonGeoDatasetSampler.split_by_percent(myDataSet, 0.8)

# 随机提取20张图生成新样本集
test_dataset = RandomNonGeoDatasetSampler.sample_by_count(myDataSet, 20) 


# Model

In [15]:
# model
from aietorch.trainer.mmseg import ChangeDetTrainer

print(ChangeDetTrainer.list_config())

['hrnet_w18_base_150k_new512_cosine_lr_batch_48_v25', 'hrnet_w18_base_150k_new512_cosine_lr_batch_48_v25_semi', 'effi-b0_base_50k_new256_cosine_lr_batch_128_adamw', 'hrnet_w18_base_150k_new512_cosine_lr_batch_48_v25_finetune']


In [16]:
trainer = ChangeDetTrainer(work_dir=work_dir, config_name="hrnet_w18_base_150k_new512_cosine_lr_batch_48_v25")

# Customized Model

In [18]:
# use custom model
from aietorch.models.changedet import ChangedetEncoderDecoder
from aietorch.engine.mmseg.models.builder import SEGMENTORS

@SEGMENTORS.register_module
class MyChangedetEncoder(ChangedetEncoderDecoder):
    pass

trainer.cfg.model.type = "MyChangedetEncoder"

# Trainer Setting

In [19]:
# 设置样本集
trainer.setup_dataset(myDataSet)
trainer.setup_dataset(train_dataset)
trainer.setup_dataset(val_dataset, data_type="val")
trainer.setup_dataset(test_dataset, data_type="test")

train
train
val
test


In [20]:
# 设置训练参数
trainer.cfg.runner.max_iters = 2
trainer.cfg.checkpoint_config.interval = 1
#trainer.cfg.optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
#trainer.cfg.optimizer_config = dict(type='Fp16OptimizerHook', loss_scale=512.0)

In [21]:
trainer.train(validate=False)

/Users/maomao/work/tianxun/aie-torch/aietorch/engine/mmseg/models/backbones/hrnet.py:318: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/Users/maomao/work/tianxun/aie-torch/aietorch/engine/mmseg/models/decode_heads/decode_head.py:110: UserWarning: threshold is not defined for binary, and defaultsto 0.3
  warnings.warn('threshold is not defined for binary, and defaults'


cuda available: False


2023-03-06 13:30:05,281 - mmseg - INFO - Start running, host: maomao@maomao-workspace.local, work_dir: /Users/maomao/work/tianxun/aie-torch/work_dirs/tutorial
2023-03-06 13:30:05,282 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                     

Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='MyChangedetEncoder',
    pretrained='open-mmlab://msra/hrnetv2_w18',
    backbone=dict(
        type='HRNet',
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        extra=dict(
            stage1=dict(
                num_modules=1,
                num_branches=1,
                block='BOTTLENECK',
                num_blocks=(4, ),
                num_channels=(64, )),
            stage2=dict(
                num_modules=1,
                num_branches=2,
                block='BASIC',
                num_blocks=(4, 4),
                num_channels=(18, 36)),
            stage3=dict(
                num_modules=4,
                num_branches=3,
                block='BASIC',
                num_blocks=(4, 4, 4),
                num_channels=(18, 36, 72)),
            stage4=dict(
                num_modules=3,
                num_branches=4,
                block='BA

/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: Use

In [22]:
checkpoint = os.path.join(work_dir, "latest.pth")

# Test

In [23]:
trainer.test(checkpoint, output_dir="test_ret")

load checkpoint from local path: ./work_dirs/tutorial/latest.pth
"PALETTE" not found in meta, use dataset.PALETTE instead
Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='MyChangedetEncoder',
    pretrained=None,
    backbone=dict(
        type='HRNet',
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        extra=dict(
            stage1=dict(
                num_modules=1,
                num_branches=1,
                block='BOTTLENECK',
                num_blocks=(4, ),
                num_channels=(64, )),
            stage2=dict(
                num_modules=1,
                num_branches=2,
                block='BASIC',
                num_blocks=(4, 4),
                num_channels=(18, 36)),
            stage3=dict(
                num_modules=4,
                num_branches=3,
                block='BASIC',
                num_blocks=(4, 4, 4),
                num_channels=(18, 36, 72)),
            stage4

/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/__init__.py:20: Use

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 20/20, 0.7 task/s, elapsed: 27s, ETA:     0s

# Onnx Export

In [26]:
onnx_path = checkpoint.replace(".pth", ".onnx")

trainer.export_onnx(output_file=onnx_path, checkpoint_path=checkpoint,shape=(256, 256))

load checkpoint from local path: ./work_dirs/tutorial/latest.pth


/Users/maomao/work/tianxun/aie-torch/venv/lib/python3.9/site-packages/mmcv/onnx/symbolic.py:481: UserWarning: DeprecationWarning: This function will be deprecated in future. Welcome to use the unified model deployment toolbox MMDeploy: https://github.com/open-mmlab/mmdeploy
  warnings.warn(msg)
/Users/maomao/work/tianxun/aie-torch/aietorch/models/changedet/segmentors/change_detector.py:292: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if return_loss:
/Users/maomao/work/tianxun/aie-torch/aietorch/engine/mmseg/ops/wrappers.py:48: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not general

Successfully exported ONNX model: ./work_dirs/tutorial/latest.onnx
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                    ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add                │ 440            │ 420              │
│ BatchNormalization │ 1              │ 1                │
│ Cast               │ 58             │ 0                │
│ Concat             │ 189            │ 5                │
│ Constant           │ 919            │ 0                │
│ Conv               │ 663            │ 663              │
│ ConvTranspose      │ 2              │ 2                │
│ Div                │ 20             │ 0                │
│ Gather             │ 164            │ 0                │
│ Identity           │ 610            │ 0                │
│ MaxPool            │ 10             │ 10               │
│ Mul                │ 48             │ 48               │
│

'./work_dirs/tutorial/latest.onnx'